In [1]:
# === Entorno del proyecto ===
import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

REPO_ROOT = Path(r"C:\Users\manue\OneDrive\Escritorio\Proyecto WhoScored")
SRC = REPO_ROOT / "src"
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

from whoscored_viz import paths, identity

# Asegura BASE_DIR / ESCUDOS_DIR si el notebook se ejecuta suelto
paths.BASE_DIR = Path(r"C:\Users\manue\OneDrive\Escritorio\Proyecto WhoScored\data\raw\matchcenter\MatchCenter")
paths.ESCUDOS_DIR = Path(r"C:\Users\manue\OneDrive\Escritorio\Proyecto WhoScored\assets\Escudos\LaLiga")
paths.OUT_DIR = REPO_ROOT / r"data\dictionaries"

TEAM_CSV = paths.TEAM_CSV
PLAYERS_CSV = paths.PLAYERS_CSV

print("[TEAM_CSV]", TEAM_CSV)
print("[ESCUDOS_DIR]", paths.ESCUDOS_DIR)

[paths.py] PROJECT_ROOT: c:\Users\manue\OneDrive\Escritorio\Proyecto WhoScored
[paths.py] BASE_DATA_DIR: C:\Users\manue\OneDrive\Escritorio\Proyecto WhoScored\data
[TEAM_CSV] C:\Users\manue\OneDrive\Escritorio\Proyecto WhoScored\data\dictionaries\team_identity.csv
[ESCUDOS_DIR] C:\Users\manue\OneDrive\Escritorio\Proyecto WhoScored\assets\Escudos\LaLiga


In [2]:
from mplsoccer import Pitch

BG_COLOR   = "#24282a"
LINE_COLOR = "white"

def make_pitch(figsize=(10,7)):
    fig, ax = plt.subplots(figsize=figsize)
    fig.patch.set_facecolor(BG_COLOR)
    ax.patch.set_facecolor(BG_COLOR)
    pitch = Pitch(
        pitch_type="uefa",
        corner_arcs=True,
        pitch_color=BG_COLOR,
        line_color=LINE_COLOR,
        linewidth=2,
        stripe=False
    )
    pitch.draw(ax=ax)
    return fig, ax, pitch

In [3]:
# === Mapeo id -> estilo de equipo desde identity/TEAM_CSV ===

# Carga del maestro de equipos (ya generado con Identidad.ipynb)
teams_master = pd.read_csv(TEAM_CSV)

def team_style_ex(team_id: int, fallback_name: str = "") -> dict:
    """
    Devuelve dict con: primary, secondary, logo, slug, name
    Lee de identity.team_style(), que ya usa TEAM_CSV + ESCUDOS_DIR.
    """
    return identity.team_style(int(team_id), fallback_name=fallback_name)

def team_color(team_id: int, kind: str = "primary", fallback: str = "#9E9E9E") -> str:
    s = team_style_ex(team_id)
    return s.get(kind) or fallback

def team_logo(team_id: int) -> str | None:
    s = team_style_ex(team_id)
    return s.get("logo")

def team_name(team_id: int) -> str:
    s = team_style_ex(team_id)
    return s.get("name") or str(team_id)

In [4]:
from whoscored_viz.utils_io import read_csv_safe

def load_match_meta_csv(match_dir: Path) -> dict:
    """
    match_dir: carpeta del partido que contiene 'csv/match_meta.csv'
    Devuelve dict con: match_id, home_id, home_name, away_id, away_name
    """
    mm = read_csv_safe(match_dir / "csv" / "match_meta.csv")
    if mm is None or mm.empty:
        raise FileNotFoundError("match_meta.csv no encontrado o vacío en " + str(match_dir))

    cols = {c.strip().lower(): c for c in mm.columns}
    def col(*names):
        for n in names:
            if n in cols: return cols[n]
    hid = col("home_team_id","home_id","hometeamid")
    hnm = col("home_name","home_team_name","hometeamname","home")
    aid = col("away_team_id","away_id","awayteamid")
    anm = col("away_name","away_team_name","awayteamname","away")
    mid = col("match_id","matchid","id")

    return dict(
        match_id = int(mm.iloc[0][mid]),
        home_id  = int(mm.iloc[0][hid]),
        home_name= str(mm.iloc[0][hnm]),
        away_id  = int(mm.iloc[0][aid]),
        away_name= str(mm.iloc[0][anm]),
    )

# === EJEMPLO: apunta a la carpeta del partido que estés visualizando
match_dir = Path(r"C:\Users\manue\OneDrive\Escritorio\Proyecto WhoScored\data\raw\matchcenter\MatchCenter\Competition\Season\20250815_Girona_vs_Rayo_Vallecano_1913916")
meta = load_match_meta_csv(match_dir)

H_ID, A_ID = meta["home_id"], meta["away_id"]
H_NAME, A_NAME = meta["home_name"], meta["away_name"]
H_COL, A_COL = team_color(H_ID, "primary"), team_color(A_ID, "primary")
H_LOGO, A_LOGO = team_logo(H_ID), team_logo(A_ID)

meta

{'match_id': 1913916,
 'home_id': 2783,
 'home_name': 'Girona',
 'away_id': 64,
 'away_name': 'Rayo Vallecano'}

In [5]:
import matplotlib.image as mpimg
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

def add_logo(ax, logo_path: str | None, xy: tuple[float,float], zoom: float = 0.12):
    if not logo_path:
        return
    try:
        arr = mpimg.imread(logo_path)
        ab = AnnotationBbox(OffsetImage(arr, zoom=zoom), xy, frameon=False, box_alignment=(0.5,0.5))
        ax.add_artist(ab)
    except Exception:
        pass

In [ ]:
def plot_shotmap(ax, Shotsdf, hteamName, ateamName, hcol, acol):
    pitch = Pitch(pitch_type="uefa", corner_arcs=True, pitch_color=bg_color, linewidth=2, line_color=line_color)
    pitch.draw(ax=ax)
    ax.set_ylim(-0.5, 68.5)
    ax.set_xlim(-0.5, 105.5)

    # Helpers
    def is_bigc(s: pd.Series) -> pd.Series:
        return s.str.contains("BigChance", na=False)

    def sel(df, team, typ, bigc=False):
        q = (df["teamName"] == team) & (df["type"] == typ)
        return df[q & (is_bigc(df["qualifiers"]) if bigc else ~is_bigc(df["qualifiers"]))]

    # HOME sin BigChance
    hGoal  = sel(Shotsdf, hteamName, "Goal",        bigc=False)
    hPost  = sel(Shotsdf, hteamName, "ShotOnPost",  bigc=False)
    hSave  = sel(Shotsdf, hteamName, "SavedShot",   bigc=False)
    hMiss  = sel(Shotsdf, hteamName, "MissedShots", bigc=False)

    # HOME BigChance
    bc_hGoal = sel(Shotsdf, hteamName, "Goal",        bigc=True)
    bc_hPost = sel(Shotsdf, hteamName, "ShotOnPost",  bigc=True)
    bc_hSave = sel(Shotsdf, hteamName, "SavedShot",   bigc=True)
    bc_hMiss = sel(Shotsdf, hteamName, "MissedShots", bigc=True)

    # AWAY sin BigChance
    aGoal  = sel(Shotsdf, ateamName, "Goal",        bigc=False)
    aPost  = sel(Shotsdf, ateamName, "ShotOnPost",  bigc=False)
    aSave  = sel(Shotsdf, ateamName, "SavedShot",   bigc=False)
    aMiss  = sel(Shotsdf, ateamName, "MissedShots", bigc=False)

    # AWAY BigChance
    bc_aGoal = sel(Shotsdf, ateamName, "Goal",        bigc=True)
    bc_aPost = sel(Shotsdf, ateamName, "ShotOnPost",  bigc=True)
    bc_aSave = sel(Shotsdf, ateamName, "SavedShot",   bigc=True)
    bc_aMiss = sel(Shotsdf, ateamName, "MissedShots", bigc=True)

    # --- DIBUJO (mismo criterio que la original, con markers por outcome) ---
    # HOME normal (coordenadas espejo para atacar a derecha)
    pitch.scatter((105-hGoal.x), (68-hGoal.y), s=250, edgecolors=hcol, c="None", marker="o", ax=ax)
    pitch.scatter((105-hPost.x), (68-hPost.y), s=250, edgecolors=hcol, c=hcol, marker="o", ax=ax)
    pitch.scatter((105-hSave.x), (68-hSave.y), s=250, edgecolors=hcol, c="None", hatch="///////", marker="o", ax=ax)
    pitch.scatter((105-hMiss.x), (68-hMiss.y), s=250, edgecolors=hcol, c="None", marker="o", ax=ax)

    # HOME Big Chances (más grandes)
    pitch.scatter((105-bc_hGoal.x), (68-bc_hGoal.y), s=500, edgecolors="green", linewidths=0.6, c="None", marker="football", ax=ax)
    pitch.scatter((105-bc_hPost.x), (68-bc_hPost.y), s=500, edgecolors=hcol, c=hcol, marker="o", ax=ax)
    pitch.scatter((105-bc_hSave.x), (68-bc_hSave.y), s=500, edgecolors=hcol, c="None", hatch="///////", marker="o", ax=ax)
    pitch.scatter((105-bc_hMiss.x), (68-bc_hMiss.y), s=500, edgecolors=hcol, c="None", marker="o", ax=ax)

    # AWAY normal (sin espejo: atacan a izquierda)
    pitch.scatter(aGoal.x, aGoal.y, s=250, edgecolors=acol, c="None", marker="o", ax=ax)
    pitch.scatter(aPost.x, aPost.y, s=250, edgecolors=acol, c=acol, marker="o", ax=ax)
    pitch.scatter(aSave.x, aSave.y, s=250, edgecolors=acol, c="None", hatch="///////", marker="o", ax=ax)
    pitch.scatter(aMiss.x, aMiss.y, s=250, edgecolors=acol, c="None", marker="o", ax=ax)

    # AWAY Big Chances
    pitch.scatter(bc_aGoal.x, bc_aGoal.y, s=500, edgecolors="green", linewidths=0.6, c="None", marker="football", ax=ax)
    pitch.scatter(bc_aPost.x, bc_aPost.y, s=500, edgecolors=acol, c=acol, marker="o", ax=ax)
    pitch.scatter(bc_aSave.x, bc_aSave.y, s=500, edgecolors=acol, c="None", hatch="///////", marker="o", ax=ax)
    pitch.scatter(bc_aMiss.x, bc_aMiss.y, s=500, edgecolors=acol, c="None", marker="o", ax=ax)

    ax.set_title("Shot Map (FotMob) — Big Chances diferenciadas", color="white", pad=10)

In [ ]:
def plot_goalPost(ax, Shotsdf, hteamName, ateamName, hcol, acol):
    # Dibuja la portería derecha (home) y la izquierda (away) con los impactos
    # Reusa el mismo pitch de fondo para mantener coherencia visual
    pitch = Pitch(pitch_type="uefa", corner_arcs=True, pitch_color=bg_color, linewidth=2, line_color=line_color)
    pitch.draw(ax=ax)
    ax.set_ylim(-0.5, 68.5)
    ax.set_xlim(-0.5, 105.5)

    # HOME: poste/portería (espejo a derecha)
    hPost  = Shotsdf[(Shotsdf["teamName"]==hteamName) & (Shotsdf["type"]=="ShotOnPost")]
    pitch.scatter((105-hPost.x), (68-hPost.y), s=220, edgecolors=hcol, c=hcol, marker="o", ax=ax)

    # AWAY: poste/portería (a izquierda)
    aPost  = Shotsdf[(Shotsdf["teamName"]==ateamName) & (Shotsdf["type"]=="ShotOnPost")]
    pitch.scatter(aPost.x, aPost.y, s=220, edgecolors=acol, c=acol, marker="o", ax=ax)

    ax.set_title("Impactos en Poste/Portería", color="white", pad=10)

In [ ]:
import matplotlib.pyplot as plt
# === DIBUJO ===
# Por fidelidad al original: usamos el shotmap de FotMob
Shotsdf = shots_df_fm.copy()   # FotMob
fig, ax = plt.subplots(figsize=(10, 7))
fig.patch.set_facecolor(bg_color)
plot_shotmap(ax, Shotsdf, hteamName, ateamName, hcol, acol)
plt.show()

# (opcional) detalle poste/portería
fig, ax = plt.subplots(figsize=(10, 7))
fig.patch.set_facecolor(bg_color)
plot_goalPost(ax, Shotsdf, hteamName, ateamName, hcol, acol)
plt.show()